In [96]:
# ADHD Classification Based on HRV
# Purpose: Determine whether heart rate variability can predict diagnosis of ADHD
# Author: Alexander Maksiaev

In [97]:
# Housekeeping

import pandas as pd
import numpy as np
import sktime.utils
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# from tsfresh import extract_features
# from tsfresh import extract_relevant_features
# from tsfresh import select_features
# import featuretools as ft 
# from featuretools.selection import (
#     remove_highly_correlated_features,
#     remove_highly_null_features,
#     remove_single_value_features,
# )
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sktime.pipeline import make_pipeline
from sktime.transformations.panel.catch22 import Catch22
from sklearn.metrics import accuracy_score
import glob
import os
import csv 
from datetime import datetime

# Directories

adhd_dir_raw = "C:/Users\maksi\Documents\Statistics\Projects\Movement_Mental_Health\hyperaktiv\hyperaktiv_with_controls\hyperaktiv_with_controls"
adhd_dir = adhd_dir_raw.replace("\\", "/")
print(adhd_dir)
activity_dir = os.path.join(adhd_dir, "activity_data")
hrv_dir = os.path.join(adhd_dir, "hrv_data")
print(hrv_dir)
output_dir = os.path.join(adhd_dir, "pt_features") 
# controls_dir = os.path.join(adhd_dir, "hyperaktiv_with_controls/hyperaktiv_with_controls/")
# activity_dir_controls = os.path.join(controls_dir, "activity_data/")

os.chdir(adhd_dir)

# Main files
patient_info = pd.read_csv("patient_info.csv", delimiter=";")
# features = pd.read_csv("features.csv", delimiter=";")
patient_info['ID'] = patient_info['ID'].astype("string").str.zfill(2)
patient_info
# print(features)

# os.chdir(activity_dir)
# activity_data_01 = pd.read_csv("patient_activity_01.csv", delimiter=";")
# activity_data_01["ID"] = 1
# activity_data_01

# os.chdir(hrv_dir)
# hrv_data = pd.read_csv("patient_hr_1.csv", delimiter=";")
# hrv_data

C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls
C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\hrv_data


,ID,SEX,AGE,ACC,ACC_TIME,ACC_DAYS,HRV,HRV_TIME,HRV_HOURS,CPT_II,...,HADS_D,MED,MED_Antidepr,MED_Moodstab,MED_Antipsych,MED_Anxiety_Benzo,MED_Sleep,MED_Analgesics_Opioids,MED_Stimulants,filter_$
0,01,0,3,1,16:00:00,6,1,11:00:00,21.0,0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,02,0,4,1,10:54:00,6.8,0,NaN,NaN,1,...,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,03,1,2,1,15:28:00,7.2,1,15:25:00,21.0,1,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,04,1,3,0,NaN,NaN,1,16:55:00,22.0,1,...,6.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,0.0
4,05,1,1,1,14:24:00,5.9,1,16:00:00,12.0,1,...,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,236,1,2,1,9:30:00,"13,3",0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,237,0,1,1,15:00:00,14,0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,238,0,4,1,9:00:00,"14,6",0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,239,0,4,1,9:00:00,"14,3",0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Activity data feature extraction using sktime 

def read_activity_file(filepath, patient_id):
    data = [ ]
    with open(filepath) as f:
        csv_reader = csv.reader(f, delimiter=";")
        next(csv_reader)
        for line in csv_reader:
            data.append([ datetime.strptime(line[0], "%m-%d-%Y %H:%M").timestamp(), int(line[1].split(" ")[0])])
    data = pd.DataFrame(data, columns=["TIME", "ACC"])
    data["ID"] = patient_id
    return data

all_participants = pd.DataFrame()
for filepath in glob.glob(os.path.join(activity_dir, "*.csv")):
    print("Reading %s" % filepath)
    patient_id_raw = os.path.splitext(os.path.basename(filepath))[0]
    patient_id = patient_id_raw.split("_")[-1]
    feature_filepath = os.path.join(output_dir, "%s_features.csv" % patient_id)

    if os.path.exists(feature_filepath):
        print("Skipped...")
        continue
    
    data = read_activity_file(filepath, patient_id)
    data["TIME"] = pd.to_datetime(data["TIME"])
    adhd_or_no = patient_info[patient_info["ID"] == patient_id]["ADHD"].iloc[0]
    print(adhd_or_no)
    data["ADHD"] = adhd_or_no
    # features = extract_features(data, column_id="ID", column_value="ACC", column_sort="TIME", n_jobs=0, show_warnings=False)
    
    # data_transposed = data.T
    
    # data.index = data["TIME"]
    # data.index.name = "TIME"
    new_data = data.loc[:, ["ACC", "ADHD"]]
    data_multi = pd.MultiIndex.from_product([[patient_id], data["TIME"]], names=["ID_index", "TIME"]) 
    new_data.index = data_multi
    
    # all_participants = pd.concat([all_participants, new_data])
    

# print(arrays)
# tuples = list(zip(*arrays))
# data_multi = pd.MultiIndex.from_product((patient_id), names=["ID"]) # , "TIME", "ACC"])

Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\activity_data\patient_activity_01.csv
1
Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\activity_data\patient_activity_02.csv
1
Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\activity_data\patient_activity_03.csv
1
Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\activity_data\patient_activity_05.csv
1
Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\activity_data\patient_activity_07.csv
0
Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyp

In [ ]:
all_participants.index = pd.MultiIndex.from_tuples(all_participants.index, names=["instances", "timepoints"])
# print(all_participants)

# X is the data to classify, y is the ground truth 

# diagnoses = patient_info[["ID", "ADHD"]]
# # diagnoses_transposed = diagnoses.T
# # print(diagnoses_transposed)
# # y = pd.concat([all_participants, diagnoses])
# all_participants_index = all_participants.index
# all_participants = pd.merge(all_participants, diagnoses, left_on="instances", right_on="ID") # , right_on="ID")
# y.index = all_participants_index
# print(all_participants)

all_participants = sktime.datatypes.convert_to(all_participants.sort_index(), to_type="pd-multiindex")



                                         ACC  ADHD
instances timepoints                              
01        1970-01-01 00:00:01.235422800    0     1
          1970-01-01 00:00:01.235422860  195     1
          1970-01-01 00:00:01.235422920  240     1
          1970-01-01 00:00:01.235422980  209     1
          1970-01-01 00:00:01.235423040  202     1
...                                      ...   ...
98        1970-01-01 00:00:01.297873620  209     1
          1970-01-01 00:00:01.297873680   43     1
          1970-01-01 00:00:01.297873740   59     1
          1970-01-01 00:00:01.297873800   12     1
          1970-01-01 00:00:01.297873860    0     1

[1494322 rows x 2 columns]
True
                                          ACC  ADHD
instances timepoints                               
228       1970-01-01 00:00:01.074410160    22     0
214       1970-01-01 00:00:01.048120860  4201     0
33        1970-01-01 00:00:01.255536000    16     0
63        1970-01-01 00:00:01.273631640     

TypeError: X must be in an sktime compatible format. Allowed scitypes for classifiers are Panel mtypes, for instance a pandas.DataFrame with MultiIndex and last(-1) level an sktime compatible time index. Allowed compatible mtype format specifications are: ['nested_univ', 'numpy3D', 'numpyflat', 'pd-multiindex', 'pd-wide', 'pd-long', 'df-list', 'gluonts_ListDataset_panel', 'gluonts_PandasDataset_panel', 'polars_panel'] . See the data format tutorial examples/AA_datatypes_and_datasets.ipynb. If you think the data is already in an sktime supported input format, run sktime.datatypes.check_raise(data, mtype) to diagnose the error, where mtype is the string of the type specification you want. Error message for checked mtypes, in format [mtype: message], as follows: [dask_panel: obj must be of python type dask.dataframe, or a subtype thereof, but found <class 'pandas.core.frame.DataFrame'>]  [df-list: obj must be of python type list, or a subtype thereof, but found <class 'pandas.core.frame.D

In [108]:
print(all_participants)
print(sktime.datatypes.check_raise(all_participants, mtype="pd-multiindex"))

X_train, X_test, y_train, y_test = train_test_split(all_participants, all_participants["ADHD"], test_size=0.33, random_state=42)

catch22 = Catch22()
randf = RandomForestClassifier(n_estimators=100)
pipe = make_pipeline(catch22, randf)

print(X_train)
print(sktime.datatypes.check_raise(X_train.sort_index(), mtype="pd-multiindex"))

# transformed_data_mv = catch22.fit_transform(X_train.sort_index())
# transformed_data_mv.head()

pipe.fit(X_train.sort_index(), y_train.sort_index())
y_pred = pipe.predict(X_test.sort_index())

accuracy_score(y_test.sort_index(), y_pred)

# features.to_csv(feature_filepath, index=False, sep=";")   

                                         ACC  ADHD
instances timepoints                              
01        1970-01-01 00:00:01.235422800    0     1
          1970-01-01 00:00:01.235422860  195     1
          1970-01-01 00:00:01.235422920  240     1
          1970-01-01 00:00:01.235422980  209     1
          1970-01-01 00:00:01.235423040  202     1
...                                      ...   ...
98        1970-01-01 00:00:01.297873620  209     1
          1970-01-01 00:00:01.297873680   43     1
          1970-01-01 00:00:01.297873740   59     1
          1970-01-01 00:00:01.297873800   12     1
          1970-01-01 00:00:01.297873860    0     1

[1494322 rows x 2 columns]
True
                                          ACC  ADHD
instances timepoints                               
228       1970-01-01 00:00:01.074410160    22     0
214       1970-01-01 00:00:01.048120860  4201     0
33        1970-01-01 00:00:01.255536000    16     0
63        1970-01-01 00:00:01.273631640     

ValueError: Mismatch in number of cases. Number in X = 116 nos in y = 1001195

In [ ]:
# # Data initialization

# os.chdir(adhd_dir)

# controls_info = pd.read_csv("patient_info.csv", delimiter=";") # Includes patients as well
# # print(controls_info)

# adhd_patients = controls_info[controls_info["ADHD"] == 1.0] # ADHD only
# # print(adhd_patients)
# # print(controls_info.columns)

# controls = controls_info[np.sum(controls_info.loc[:, "ADHD":"OTHER"], axis=1) == 0] # Neurotypicals only
# non_adhd_controls = controls_info[controls_info["ADHD"] == 0] # Non-ADHD only
# # print(controls)
# # print(non_adhd_controls)


In [ ]:
# # Extract HRV features for each participant

# os.chdir(hrv_dir)

# def read_hrv_file(filepath, patient_id):
#     data = [ ]
#     with open(filepath) as f:
#         csv_reader = csv.reader(f, delimiter=";")
#         next(csv_reader)
#         for line in csv_reader:
#             data.append([ datetime.strptime(line[0], "%Y-%m-%d %H:%M:%S.%f").timestamp(), float(line[1].split(" ")[0])])
#     data = pd.DataFrame(data, columns=["TIME", "HRV"])
#     data["ID"] = patient_id
#     return data

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# for filepath in glob.glob(os.path.join(hrv_dir, "*.csv")):
#     print("Reading %s" % filepath)
#     patient_id = os.path.splitext(os.path.basename(filepath))[0]
#     feature_filepath = os.path.join(output_dir, "%s_features.csv" % patient_id)

#     if os.path.exists(feature_filepath):
#         print("Skipped...")
#         continue
    
#     data = read_hrv_file(filepath, patient_id)

#     es = ft.EntitySet(id="pt_data")
#     es = es.add_dataframe(
#     dataframe_name="pt_data",
#     dataframe=data,
#     )
#     rolling_mean_primitive = RollingMean(
#         window_length=window_length, gap=gap, min_periods=window_length
#     )

#     rolling_min_primitive = RollingMin(
#         window_length=window_length, gap=gap, min_periods=window_length
#     )
#     feature_matrix, feature_defs = ft.dfs(
#     entityset=es,
#     target_dataframe_name="pt_data",
#     trans_primitives=(
#         datetime_primitives
#         + delaying_primitives
#         + [rolling_mean_primitive, rolling_min_primitive]
#     ),
#     )
#     print(feature_defs)
#     print(feature_matrix)
#     break 
#     # features.to_csv(feature_filepath, index=False, sep=";")    

Reading C:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/hyperaktiv/hyperaktiv_with_controls/hyperaktiv_with_controls\hrv_data\patient_hr_1.csv


c:\Users\maksi\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\entityset\entityset.py:1717: UserWarning: Using first column as index. To change this, specify the index parameter
  warnings.warn(
c:\Users\maksi\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
c:\Users\maksi\AppData\Local\Programs\Python\Python311\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['max', 'mean', 'min', 'skew', 'std']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.


[<Feature: HRV>, <Feature: ID>]
                  HRV            ID
TIME                               
1.236182e+09  2294.92  patient_hr_1
1.236182e+09   631.84  patient_hr_1
1.236182e+09   624.02  patient_hr_1
1.236182e+09   636.72  patient_hr_1
1.236182e+09   625.98  patient_hr_1
...               ...           ...
1.236258e+09   671.88  patient_hr_1
1.236258e+09   669.92  patient_hr_1
1.236258e+09   670.90  patient_hr_1
1.236258e+09   662.11  patient_hr_1
1.236258e+09   672.85  patient_hr_1

[107474 rows x 2 columns]


In [ ]:
# Combine feature csvs into one file for analysis 

In [ ]:
# # Concat all activity dataframes

# os.chdir(hrv_dir)

# csv_files = glob.glob("*.csv")

# # Create an empty dataframe to store the combined data
# # mega_activity_df = pd.DataFrame()

# # Loop through each CSV file and append it to the combined dataframe
# def get_activity_file(file):
# # for file in csv_files:
#     file_name_pieces = str.split(file, sep="_")
#     pt_id = file_name_pieces[-1][:-4] # Last part of the file name is patient id, without the .csv
#     df = pd.read_csv(file, delimiter=";")
#     df["ID"] = int(pt_id)
#     # mega_activity_df = pd.concat([mega_activity_df, df], ignore_index=True)
#     return df

# # mega_activity_df
# # print(len(mega_activity_df))
# # mega_activity_df = mega_activity_df.dropna()

# # print(len(mega_activity_df[mega_activity_df["ID"] <= 3]))
# activity_01 = get_activity_file(csv_files[0])
# print(activity_01)

In [ ]:
# Pre-processing: Activity

# features = extract_features(activity_01, column_id="ID", column_value="ACTIVITY", column_sort="TIMESTAMP", n_jobs=0, show_warnings=False)
# print(features.head())
# os.chdir(main_dir)
# features.to_csv("adhd_activity_features.csv", index=False, sep=",")    

# # Pts with motor activity recordings
# active_controls = controls_info[controls_info["ACC"] == 1]
# # print(active_controls)

# # Each pt has their own csv file...
# # demographics_01 = active_controls[active_controls["ID"] == 1]
# y = pd.Series(active_controls["ADHD"], index=active_controls["ID"])
# y = y.fillna(0)
# y["ADHD"] = active_controls["ADHD"]
# y.index = active_controls["ID"]
# y["ID"] = y.index
# print(y)
# print(list(y.index))
# print(demographics_01)
# print(activity_data_01)
# print(y.loc[1:3])

# activity_data_01.plot(figsize=(10,10))
# plt.show()

# Find relevant features using tsfresh
# relevant_features = extract_relevant_features(mega_activity_df[mega_activity_df["ID"] <= 3], y.loc[1:3], column_id='ID', column_sort='TIMESTAMP')

# relevant_features.head()

# X_full_train, X_full_test, y_train, y_test = train_test_split(relevant_features, y, test_size=.3, random_state=42)
# X_filtered_train = select_features(X_full_train, y_train)
# X_filtered_train.head()

In [ ]:
# # Splitting into training set and testing set

# # Don't let the machine know which ones are ADHD and which ones aren't
# X = controls_info.drop("ADHD", axis=1)
# y = controls_info["ADHD"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify = y, random_state=2025)
